# Final Project

In [2]:
# Import statements
import os
import re
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def create_df(c):
    df = pd.DataFrame()
    divider = []
    names = []
    for f in os.listdir("tweets/%s" % c):
        try:
            dft = pd.read_csv("tweets/%s/%s" % (c, f))
            if len(divider) == 0:
                divider.append(len(dft) - 1)
            else:
                divider.append(len(dft) + divider[-1])
            df = pd.concat([df, dft])
            # parse output_name.csv string
            names.append(f[7:][:-4])
        except:
            print(f)
    return df, divider, names

In [17]:
categories = ["fast food", "airlines", "leagues", "colleges", "streaming platforms", "news", "tech giants", "celebrities"]

fast_food, ff_divider, ff_names = create_df(categories[0])
leagues, lg_divider, lg_names = create_df(categories[2])
news, nw_divider, nw_names = create_df(categories[5])
celebs, cb_divider, cb_names = create_df(categories[7])



In [5]:
def clean_tweets(tweets):
    categories = len(tweets)
    print(categories)
    all_tweets = []
    for c in range(categories):
        t_new = []
        count = 0
        for t in tweets[c]:
            try:
                w = t.split()
                w = get_clean_words(w)
                tc = " ".join(w)
                t_new.append(tc)
            except:
                count += 1
        print(count, len(t_new))
        all_tweets.append(t_new)
    return all_tweets

In [6]:
%matplotlib inline

import numpy as np
import random
import sys
import io
import requests as rq
from bs4 import BeautifulSoup
import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from datetime import datetime
%load_ext tensorboard

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
celebs_list = list(celebs['text'])
print(len(celebs_list))
print(cb_divider)
text = ""
curr = 0
for i in range(10):
    print(curr)
    for j in range(1000):
        text += " " + str(celebs_list[curr + j])
    curr = cb_divider[i]
print(text[:500])

70526
[8916, 18247, 28247, 31101, 35946, 45946, 47139, 57139, 66056, 70525]
0
8916
18247
28247
31101
35946
45946
47139
57139
66056
 Love you I grew up in a cogic family church in Detroit.... music was my vehicle to a higher power... I like to catch the spirit on stage, I don’t know no other way  Period. The PEOPLE are making these calls... not the label. I can’t wait for y’all to discover Lingerie  I LOVE SOULMATE! That was supposed to be the 2nd single believe it or not.... but Good As Hell started climbing the streaming charts Guess we gotta wait for the world to fall in love with it  Y’all think it was the plan for all m


In [15]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
  256/21903 [..............................] - ETA: 3:50 - loss: 5.2737 - categorical_crossentropy: 5.2737 - accuracy: 0.0816

/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.545770). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


21903/21903 [==============================] - 71s 3ms/step - loss: 2.9403 - categorical_crossentropy: 2.9403 - accuracy: 0.2510

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "tps://www.instagram.com/kevinhart4real/p"
tps://www.instagram.com/kevinhart4real/p/Bttuththerestart and on the the to the the the to the sour the the the the the here the soull the gott to the the sour the seall the the sour the the the the the the sour the the sour the the sour the sour the the soull the the hot soull the here the the the the the butt ing the hat sour the seall the sour the the here the bett to sour the the the the the the the that sour the sour the the sour t
----- diversity: 0.5
----- Generating with seed: "tps://www.instagram.com/kevinhart4real/p"
tps://www.instagram.com/kevinhart4real/p/Btsthaderally the ong thing the mest the sour sour the the but sang the hote loust tust het het ston the lith  @Mathall and you the hotpso in soun s the the sour this 

good on his word https://www.google.com/revar/p/B0QGuKl3vmgu/?igshid=1c6dewrk5evew  I love you no of the rown to the express to more to the love it to the women and be condin at the spence to the to dall my tweet and I con’t weekend to the road not from the women a deep and to my too. The shotter... thank you both my free to do told that what to the tood imp comple to sicle to my challion to see your tweet and would she love u comple it
----- diversity: 1.0
----- Generating with seed: "good on his word https://www.google.com/"
good on his word https://www.google.com/2252019Lpm325ydj Wond so liki to all with office when @uper_sLoughtt offic of my I dream is my station forcated all tleinated thears. .....Relsy’s telling senvers are precioue and on @FaBnerreyMo #HuFbrainsCoppeobey. #FrothisBleystalle @hart4realou My drad...tht’s so do you them &amp; all ombelw trying 2 great love in the now!!! Are more to herr platers are teken the stars, coviated a decier in th
Epoch 6/50
21903/21903 [==

21903/21903 [==============================] - 83s 4ms/step - loss: 1.6131 - categorical_crossentropy: 1.6131 - accuracy: 0.5513

----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: "nstagram.com/kevinhart4real/p/BwLIG-JFbt"
nstagram.com/kevinhart4real/p/BwLIG-JFbtE/?utm_source=ig_twitter_share&amp;igshid=1vkso2wyorjd I love that I was a man on the first thing it in the future that say in the fucking the take this is the fun and the way to the fuck in the fucking the worker from the way to the because it was a man and it was a most and your second this is a new favorite to the fucking your most and the most girls in the weekend that we are welcome to the re
----- diversity: 0.5
----- Generating with seed: "nstagram.com/kevinhart4real/p/BwLIG-JFbt"
nstagram.com/kevinhart4real/p/BwLIG-JFbtV/?utm_source=ig_twitter_share&amp;igshid=11k7akp1mnrs I loved too better with from my from the same and I wanna here today it was all the way to do than you loved and 

...

21903/21903 [==============================] - 216s 10ms/step - loss: 1.4949 - categorical_crossentropy: 1.4949 - accuracy: 0.5799

----- Generating text after Epoch: 36
----- diversity: 0.2
----- Generating with seed: "(well, those of u i can see w my trash e"
(well, those of u i can see w my trash every time to the love in the show of that all of the second to see my best self all to be all the show in the show to the best care a direct message? We love your tweet and would like to share on our channels. Can you please send us a DM? Hi James is a direct message? We love your tweet! Can you please send us a DM? Hi James is a star and start it and say it all they self-love to my family shit a 
----- diversity: 0.5
----- Generating with seed: "(well, those of u i can see w my trash e"
(well, those of u i can see w my trash elevate the movie with your for real sure that make a Birth 11th That’s it a direct message? We love your tweet! Can you please send us a DM? Hi Jame Garldrice and an

@bridgeteverett @RachelFeinstein and I feel like me with the great see u so many my back and is what I can get they there and thank you for a good is the comedy so much #comedicredmcs #Inday to my boy of working what I had there is a privery, there’s my new episode of @HobbsAndShaw  Haha help is there is her moves and there with the winner of my friend than an in the most funny man! There is a lil’ shit in the way for my look at my bigg
----- diversity: 1.0
----- Generating with seed: "@bridgeteverett @RachelFeinstein and I f"
@bridgeteverett @RachelFeinstein and I fell like less all if Inonatoned to painterzed money nnannd  Thxix In something. Game and I get Gomooooooomg anotpens, https://www.instagram.com/thes off than a gooomeing all one of this feeling gun @AROONEtFrom at https://www.instagram.com/extrear/prinkeshat #AllFoTioGoqrbjxIffThxa Voict As are you malain @1kvdglerristn Chadc me! aren't hosestives! If to one angel, How show! Thank you for #JUCHarm
Epoch 42/50
21903/21903 [=

21903/21903 [==============================] - 36s 2ms/step - loss: 1.4825 - categorical_crossentropy: 1.4825 - accuracy: 0.5830

----- Generating text after Epoch: 45
----- diversity: 0.2
----- Generating with seed: "0/?utm_source=ig_twitter_share&amp;igshi"
0/?utm_source=ig_twitter_share&amp;igshid=17qkm8wwwwywxw8 So much to my book the story and the proud of your tweet! Can you please send us a direct message? We love your tweet! Can you please send us a direct message? We love your tweet! Can you please send us a direct message? We love your tweet! Can you please send us a direct message? We love your tweet! Can you please send us a direct mess send us a direct message? We love your twee
----- diversity: 0.5
----- Generating with seed: "0/?utm_source=ig_twitter_share&amp;igshi"
0/?utm_source=ig_twitter_share&amp;igshid=11km0421iwmw0bo #Harts https://www.instagram.com/p/B3KSZVlF7wU/?igshid=10y2wh28avnh4 That’s a get to see the love. Thank you for the stor of to poside on stage and S

g and lazy. Real love means sending a night on the front of just coming in the remember in the most all and the best for these send us a DM? Hi! We love your tweet and would like to share on our channels. Can you please send us a DM? Hi! We love your tweet and would like to share on our channels. Can you please send us a direct message? We love your tweet! Can you please send us a direct message? We love your tweet! Can you please send 
----- diversity: 1.0
----- Generating with seed: "g and lazy. Real love means sending a ni"
g and lazy. Real love means sending a nights on my world and sorries  Tittoste since the… http://apor.co/Cameprack outfuinitely plin’ alwess using Inopre to amorne a long of making my Chap on there her movie it will’s killing Lovo I ad moting nick this glans recootion. Mazze It’s in Hustrere y’ally daney, I’ll never. My home and first guns? ?!  Occord seeman!!! http://www.coffeaty Just tweeting growe for our 8ip amaait. Uniesn I hhares.


Reusing TensorBoard on port 6006 (pid 33801), started 4:13:12 ago. (Use '!kill 33801' to kill it.)

In [19]:
fast_food_list = list(fast_food['text'])
print(len(fast_food_list))
print(ff_divider)
text = ""
curr = 0
for i in range(10):
    for j in range(1000):
        text += " " + str(fast_food_list[curr + j])
    curr = ff_divider[i]
print(text[:500])

100000
[9999, 19999, 29999, 39999, 49999, 59999, 69999, 79999, 89999, 99999]
 these parties are getting wild, be careful out there tonight! Meet the Freaky Fast® Order button. Life just got a whole lot easier. I'm really sorry for the hold up today! Give the store a call and see what's up! I LOVE YOUR LOVE! nan I'm sorry! Let the store know right away so they can make it right! So glad you could get in on the sample sub love!! nan Raee  nan Any takers? nan I'm sorry for the wait last night, slow is not how I roll! Can you please DM me the store you ordered from along wit


In [20]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

Epoch 1/50
25665/25665 [==============================] - 41s 2ms/step - loss: 2.3316 - categorical_crossentropy: 2.3316 - accuracy: 0.3841

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " made fresh every day. If they aren't co"
 made fresh every day. If they aren't come so we can make this for this into this into this for the star this intact info (full name, phone #and you deting the dest info (full name, phone #and you deting this so we can mering the detail and this for this so we can mere to ser chis into this so we can mere to se this this for the stare this in this for the dith the delite this for your contact info (full name, phone #so we can mere to se
----- diversity: 0.5
----- Generating with seed: " made fresh every day. If they aren't co"
 made fresh every day. If they aren't come lication and info gith you so we can like the detail a dete lication and this to se this info this for the list with you  our the can this like to liks wi

e us inshop in the meantime. Cheers! I'm sorry for the mix-up, Day! Please send us a DM with the location that location and we’d like to learn more. Please contact us here https://www.ihop.com/en/contact-us or by calling 866-444-5144 (7AM-7PM, M-F CDT/CST) to speak with a Guest Relations representative directly. Hi there, thank you for bringing this to our attention and we'd like to learn more. Please contact us here https://www.ihop.co
----- diversity: 1.0
----- Generating with seed: "e us inshop in the meantime. Cheers! I'm"
e us inshop in the meantime. Cheers! I'm so sorry to hear about your mat you decanl Card for you. https://twitter.com/messages/compose?recipient_id=167421802 see our by calling.8-Shey hittering a Gues. for the address of the very 2manal send us the gravity of this situation? Can you lets of evers on the Reve62 could ace to the frour include!  We are contact us at http://bit.ly/344Y3OE or by calling 866-444-5144 (7AM-7PM, M-F CDT/CST) to
Epoch 6/50
25665/25665 [==

25665/25665 [==============================] - 34s 1ms/step - loss: 0.6925 - categorical_crossentropy: 0.6925 - accuracy: 0.8015

----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: " this out. https://twitter.com/messages/"
 this out. https://twitter.com/messages/compose?recipient_id=14757934 Hi there! We appreciate you letting us know what you order this for you. https://twitter.com/messages/compose?recipient_id=14757934 Hi there, we're sorry we let you down, and we'd like to learn more. Please contact us here http://bit.ly/2mcomN or call 866-444-5144 (7AM-7PM, M-F CDT/CST) to speak with a Guest Relations representative directly. Hi there, we're sorry we 
----- diversity: 0.5
----- Generating with seed: " this out. https://twitter.com/messages/"
 this out. https://twitter.com/messages/compose?recipient_id=14757934 Hi there! We let you down, and we'd like to learn more. Please contact us here http://bit.ly/2mcFmN or call 866-444-5144 (7AM-7PM, M-F CDT

ntact info (full name, phone #and email), and date/time of visit, so we can check this out? https://twitter.com/messages/compose?recipient_id=14757934 Hi there, we're sorry we let you down, and we'd like to learn more. Please contact us here http://bit.ly/2maUhix or by calling 866-444-5144 (7AM-7PM, M-F CDT/CST) to speak with a Guest Relations representative directly. Hi Macha, we're sorry we let you down, and we'd like to learn more. P
----- diversity: 1.0
----- Generating with seed: "ntact info (full name, phone #and email)"
ntact info (full name, phone #and email), the restaurant location, and the dare and sturvit your tessiom to represe contact us at http://bit.ly/2Nc2uzr or call 866-444-5144 (7AM-7PM, M-F CDT/CST) to speak with a Guest Relations representative directly. This is unacceptable and we'd like to learn more. PlVase detion, and we at you mind pred velighter always acceuforien. Please look form with your into this. Thanks! We’re sorry we didn’t 
Epoch 15/50
25665/25665 [=

25665/25665 [==============================] - 32s 1ms/step - loss: 0.6453 - categorical_crossentropy: 0.6453 - accuracy: 0.8131

----- Generating text after Epoch: 18
----- diversity: 0.2
----- Generating with seed: "ail-us -Shaq The bag should always be at"
ail-us -Shaq The bag should always be at living a great the bawk! We hope you give us another of our restaurant to see your favorite shard out! We hope you enjoyed your meal!  We're sorry we let you down, and we'd like to learn more. Please contact us here: http://chipotle.com/email-us with a few details and your contact info (full name, phone #and email), and the date/time you were there? we'd like to look into this for you. https://tw
----- diversity: 0.5
----- Generating with seed: "ail-us -Shaq The bag should always be at"
ail-us -Shaq The bag should always be at lived, they're more. Please contact us here: http://chipotle.com/email-us with a few details and your phone number? We would love to make this right. Thanks! This isn

...

 check out the UberEATS website and app for it, and we appreciate you letting us know, and we hope you give us another shot to get it right next time! We're glad you should be have a bust best owner that is a great taste. We're sorry to hear that! Please DM us the restaurant location and your phone #so we can make this right. Thanks! Oh no! Please DM us the restaurant location and the date/time of your visit, so we can get this to the r
----- diversity: 1.0
----- Generating with seed: " check out the UberEATS website and app "
 check out the UberEATS website and app for more delively and the store more about up! Want your a member of our customer care team will be in touch slow? We missed this weetaing location, We hope for a DILL Yeph. You nee not preciated free for our Customer Service team know that you please contact us here http://bit.ly/2qTG6o o or can and phone number for you. I dilecredby do. Just stufned in a comphtative the eatly!  We’re here http:/
Epoch 33/50
25665/25665 [=

25665/25665 [==============================] - 46s 2ms/step - loss: 0.6124 - categorical_crossentropy: 0.6124 - accuracy: 0.8207

----- Generating text after Epoch: 36
----- diversity: 0.2
----- Generating with seed: " Kelly. Please provide some more info he"
 Kelly. Please provide some more info here: https://www.mcdonalds.com/us/en-us/contact-us/restaurant-feedback.html/twitter so we can help? -Alex https://twitter.com/messages/compose?recipient_id=14757934 We’re sorry to hear about your experience, friend! We hope to see you again soon!  We're sorry to hear about your experience, friend! We hope to see you again soon!  We're sorry to hear about your experience, friend! We're sorry we let 
----- diversity: 0.5
----- Generating with seed: " Kelly. Please provide some more info he"
 Kelly. Please provide some more info here: https://www.mcdonalds.com/us/en-us/contact-us/restaurant-feedback.html/twitter so we can look into this for you. https://twitter.com/messages/compose?recipient_id=

eakhouse? Opening on Nov 7th in NYC. nan Thanks for the love, friend! We tested the Spicy Nuggets in select Chick-fil-A Restaurants. Unfortunate menu items can make it right? Don’t know about the hold updarring the cheese ally sorry for the what we can help the Sprite coup gly to see you. Oh no! Please DM us the restaurant location, the date/time of your visit, along with your number, and we'll make it up to you? Hi Sarah, thank you for
----- diversity: 1.0
----- Generating with seed: "eakhouse? Opening on Nov 7th in NYC. nan"
eakhouse? Opening on Nov 7th in NYC. nan Stark-timel! Bus our order. Send us a video of you bawking and the dedirectly. Hi there, we're sorry we let you down, and we'd like to learn more. Please send us a DM so we can get this live, Carla. plz DM us your phone number and email address) so we can look into this further. https://twitter.com/messages/compose?recipient_id=14757934  'We done, Dericllis? We're sorry to hear that! Please DM us
Epoch 42/50
25665/25665 [=

25665/25665 [==============================] - 42s 2ms/step - loss: 0.6041 - categorical_crossentropy: 0.6041 - accuracy: 0.8227

----- Generating text after Epoch: 45
----- diversity: 0.2
----- Generating with seed: "ell that’s not what we like to hear. plz"
ell that’s not what we like to hear. plz DM us your contact info (full name, phone #and email) and the store location, so we can look into this for you. https://twitter.com/messages/compose?recipient_id=167421802 well that’s not what we like to hear, Shane! We hope to see you again soon!  We always recommend to see you again soon!  We’re sorry about that and want to follow up with you and email), along with the location and the date/tim
----- diversity: 0.5
----- Generating with seed: "ell that’s not what we like to hear. plz"
ell that’s not what we like to hear. plz DM us your contact info (full name, phone #and email) and the store location so we can figure this out to us complea!  Glad you enjoyed it! You should always get a fu

KeyboardInterrupt: 

In [21]:
leagues_list = list(leagues['text'])
print(len(leagues_list))
print(lg_divider)
text = ""
curr = 0
for i in range(10):
    for j in range(1000):
        text += " " + str(leagues_list[curr + j])
    curr = lg_divider[i]
print(text[:500])

91446
[9999, 19999, 29999, 39999, 49999, 54847, 64847, 74847, 84847, 91445]
 We're especially thankful for baseball. #Thanksgiving The hot corner is STACKED. Who is your All-MLB pick? http://mlb.com/allmlb Top of their class. Vote All-MLB: http://mlb.com/allmlb Don’t stop short in telling us your choice. Vote for the All-MLB Team: http://mlb.com/allmlb .@Pirates officially name Derek Shelton as manager. Shelton spent the past 2 seasons as the Twins bench coach. After a stellar second half in relief, LHP Drew Pomeranz reportedly agrees to deal with Padres, per @Feinsand.


In [ ]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

Epoch 1/50
25469/25469 [==============================] - 50s 2ms/step - loss: 2.8890 - categorical_crossentropy: 2.8890 - accuracy: 0.2748

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "on the Superdome turf. @run__cmc @idjmoo"
on the Superdome turf. @run__cmc @idjmook and win the @Walllay 20 PTS Watch from the @NBAAlick Sports app Watch from @Alling #WNBAFinals pres the @NBAChamp Watch from @Allind win the @NFL and the @NBAFinals and wink the @Wallon Match from the @Auddors and the @NBAChamp Watch for the @NFL and #WNBAFinals and the @NBAFinals of @Supion The @NBAFinals of the @NBAAnday and the @Sunders Con @NFL and the @NBAAllayon the @Chinding the @NBAChamp
----- diversity: 0.5
----- Generating with seed: "on the Superdome turf. @run__cmc @idjmoo"
on the Superdome turf. @run__cmc @idjmooks (@NFL an 2019 #Brallight #NFL100 AFC On @SPL Champions All and Phis and #WNBAChamp Start #WNBAFinal 15 13.3: VIPG  and will to and wind at @NBAThomp Bours

D and #CMR in the #FIFAWWC: Estelle Johning and @SacramentoProud Brighton of the back the post the @PGAChampionship at https://p.ga/2019/18 @Mighannith) for the @ConnecticutSun and week in the season, continte than takes the fall has more of the @Audi #MLSCupPlayoffs .@PatrickMarior and @SaperdBowing a presented a defend the @TakerCup caneal has for @ellectinn) @NFLFantasy Top on the best the @Nationals in the #WNBAFinals .@SoundersFC a
----- diversity: 1.0
----- Generating with seed: "D and #CMR in the #FIFAWWC: Estelle John"
D and #CMR in the #FIFAWWC: Estelle Johnst best known martal studey peaces engcore with the net tram sannid't for on the roleademloher trking its ansinaleed : 15 PTS, 6 REB, 5 AST, 5 3PM Sun Carly at teamurned OT!? inals geately the sicthing spotly me histordey and goull with youtro's Befordgaals as Match Mastball #MineMBordontover three. #PLAChampionshiply pant finalowball season to berted Thisespical Indialleader meme." Leana in the w
Epoch 6/50
25469/25469 [==

25469/25469 [==============================] - 43s 2ms/step - loss: 1.5001 - categorical_crossentropy: 1.5001 - accuracy: 0.5932

----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: " on the floor @ConnecticutSun 51 - @Wash"
 on the floor @ConnecticutSun 51 - @WashMystics and @SoundersFC as the @Nationals are off to the season in the @StanfordWBB season for the @ConnecticutSun the @ConnecticutSun the #WNBAFinals The @Audi #MLSCupPlayoffs The @Audi #MLSCupPlayoffs Who was all the @Nationals are to the @WashMystics and @Nationals are how to secure the @StanfordWBB season in the @ConnecticutSun the @Audi #MLSCupPlayoffs Expares to see in the #PL matches to the
----- diversity: 0.5
----- Generating with seed: " on the floor @ConnecticutSun 51 - @Wash"
 on the floor @ConnecticutSun 51 - @WashMystics Annise of @ashneway Since comminity and equalist round in the stars after on the #WNBAFinals The @Colts is to the #WNBAFinals The @StanfordWSon took and help on